In [ ]:

#  GET A CLIENT HANDLE

import os
import time

from katana import remote
from katana.remote import import_data

os.environ["KATANA_SERVER_ADDRESS"] = "localhost:8080"

my_client = remote.Client()

print(my_client)

In [ ]:

import katana

print(katana.__version__) 

In [ ]:

#  Set variables

DB_NAME = "my_db"
GRAPH_NAME = "my_graph"

print("--")

In [ ]:
#  Work in progress ..

DATABASE_ID=my_client.get_database(name=DB_NAME).database_id

GRAPH_ID = ""
GRAPH_VERSION = ""


for l_graph in my_client.graphs():
   if (l_graph.name == GRAPH_NAME):
      GRAPH_ID=l_graph.graph_id
      GRAPH_VERSION=l_graph.version
         #
      my_graph = remote.Graph(client=my_client, graph_id=GRAPH_ID, database_id=DATABASE_ID, version=GRAPH_VERSION)

print(my_graph)

In [ ]:

#  DELETE ALL GRAPHS


for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      remote.Graph(client=my_client, graph_id=l_graph.graph_id,database_id=l_database.database_id, version=l_graph.version).delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_grapg.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

In [ ]:

#  CREATE A GRAPH

my_graph=my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=2)

print(my_graph)

In [ ]:

# ##################################################################


In [ ]:

# import os
# import time
# import json

# from katana import remote
# from katana.remote import import_data
# from katana_enterprise import async_to_sync


In [ ]:

#   https://console.cloud.google.com/storage/browser/katana-demo-datasets/csv-datasets/chembl_raw

#  This is 27, we are using 29 ..

# import_data.csv(
#     my_graph,
# 
#     input_node_path="gs://katana-demo-datasets/csv-datasets/chembl_raw/mapping/node_list.txt",
#     input_edge_path="gs://katana-demo-datasets/csv-datasets/chembl_raw/mapping/edge_list.txt",
#     input_dir="gs://katana-demo-datasets/csv-datasets/chembl_raw/csv/chembl27/",
# 
#     files_have_headers=True,
#     data_delimiter="|",
#     schema_delimiter="|",
#     partition_policy="random-oec"
# 
# )

print("--")


In [ ]:

#  This is 29

%%time 

import_data.csv(
   my_graph,
    
   input_node_path="gs://katana-internal8/chemb29_rdf2lpg/lpg_chembl29/chemb29_rdf2lpg_lpg_chembl29_katana-graph-node-schema.tsv",
   input_edge_path="gs://katana-internal8/chemb29_rdf2lpg/lpg_chembl29/chemb29_rdf2lpg_lpg_chembl29_katana-graph-edge-schema.tsv",
   input_dir="gs://katana-internal8/chemb29_rdf2lpg/lpg_chembl29/",

   files_have_headers=False,
   data_delimiter="\t",
   schema_delimiter="\t",
   partition_policy="random-oec"

)

print("--")


In [ ]:
%%time

my_graph.num_nodes()

In [ ]:
%%time

my_graph.num_edges()

In [ ]:

%%time

# my_graph.schema().view()


In [ ]:

# ##################################################################


In [ ]:

#  Query 6, original 

#  %%time
#  query6='''match (b:molecule_dictionary)-[:has_molecule]-(l:activities)-[:has_assay]-(r:assays)-[:has_target]-(t:target_dictionary{target_type:'SINGLE PROTEIN'}) return count(b)'''
#  #query6='''match (b:molecule_dictionary)<-[:has_molecule]-(l:activities)-[:has_assay]->(r:assays)-[:has_target]->(t:target_dictionary{target_type:'SINGLE PROTEIN'}) return count(b)'''
#  smile6= chembl27_csv_new.query(query6)
#  print(smile6[0])


In [ ]:

#  My query 6

l_query6 = """

   match ( b:SmallMolecule ) - [ :hasMolecule ] - ( l:Activity ) - [ :hasAssay ] - ( r:Assay ) - [ :hasTarget ] - ( :SingleProtein ) 

   return count(b)
   """

l_result6 = my_graph.query(l_query6)

print(l_result6)

In [ ]:

#  Query 7, original

# %%time
# query7='''match (b:molecule_dictionary)<-[:has_molecule]-(l:activities)-[:has_assay]->(r:assays)-[:has_target]->(t:target_dictionary{target_type:'SINGLE PROTEIN', organism:'Mycobacterium tuberculosis'})  return count(b);'''
# 
# smile7= chembl27_csv_new.query(query7)
# print(smile7[0])


In [ ]:

# My query 7

l_query7="""

   match ( b:SmallMolecule ) - [ :hasMolecule ] - ( l:Activity ) - [ :hasAssay ] - ( r:Assay ) - [ :hasTarget ] - ( t:SingleProtein ) // , organism:'Mycobacterium tuberculosis'}) 
   where t.organismName = 'Mycobacterium tuberculosis'

   return count(b)
   """

l_result7 = my_graph.query(l_query7)

print(l_result7)


In [ ]:

#  Query 8, original

# %%time
# query8='''match (b:molecule_dictionary{chembl_id:'CHEMBL941'})<-[:has_molecule]-(l:activities)-[:has_assay]->(r:assays)-[:has_target]->(t:target_dictionary)<-[:has_target]-(c:target_components) return count(b);'''
# 
# smile8= chembl27_csv_new.query(query8)
# print(smile8[0])


In [ ]:

#  My query 8

l_query8 = """

   match ( b:SmallMolecule) //  - [:has_molecule] - (l:activities)-[:has_assay] -  (r:assays) - [:has_target]->(t:target_dictionary) - [:has_target]-(c:target_components) 
   
   limit 10
   return b // count(b)
   """

l_result8 = my_graph.query(l_query8)

print(l_result8)


In [ ]:

#  Query 9, original

# %%time
# query9='''match (b:molecule_dictionary)<-[:has_molecule]-(l:activities)-[:has_assay]->(r:assays)-[:has_target]->(t:target_dictionary{chembl_id_td:"CHEMBL1827"}) return count(b);'''
#query9='''match (t:target_dictionary) return (t) limit 1;'''
# smile9= chembl27_csv_new.query(query9)
# print(smile9[0])


In [ ]:

#  My query 9

l_query9 = """
   match (b:molecule_dictionary) - [:has_molecule]-(l:activities)-[:has_assay]- (r:assays)-[:has_target]->(t:target_dictionary{chembl_id_td:"CHEMBL1827"}) 
   return count(b)
   """

l_result9 = my_graph.query(l_query9)

print(l_result9)


In [ ]:

#  Query 10, original

# %%time
# query10 = '''match (f:compound_properties)-[:has_molecule]-(b:molecule_dictionary{oral:1})<-[:has_molecule]-(d:drug_indication{mesh_id:'D006333'}), (b)<-[:has_molecule]-(l:activities)-[:has_assay]->
# (r:assays)-[:has_target]->(t:target_dictionary)<-[:has_target]-(c:target_components)-[:has_comp_seq]->(o:component_sequences) where l.pchembl_value > 5.0 and f.full_mwt >= 199.0 return count(b);'''
# smile10  = chembl27_csv_new.query(query10)
# print(smile10[0])


In [ ]:

#  My query 10

l_query10 = """
   match (f:compound_properties) - [:has_molecule] - (b:molecule_dictionary{oral:1}) - [:has_molecule] - (d:drug_indication{mesh_id:'D006333'}), (b) - [:has_molecule]-(l:activities)-[:has_assay]- 
      (r:assays) - [:has_target] - (t:target_dictionary) - [:has_target] - (c:target_components) - [:has_comp_seq] - (o:component_sequences) 
   where l.pchembl_value > 5.0 and f.full_mwt >= 199.0 
   return count(b)
   """

l_result10  = my_graph.query(l_query10)

print(l_result10)

In [ ]:

# ##################################################################


In [ ]:

result = my_graph.query("""

   MATCH (n) RETURN distinct labels(n) as label, count(*)

   """)

print(result)


In [ ]:
result = my_graph.query("""

   MATCH ()-[e]->() RETURN distinct type(e) as edge_type, count(*)

   """)

print(result)

In [ ]:

# ##################################################################


In [ ]:

#  Postgres query # 1










In [33]:

l_var1 = "organismName"
l_var2 = 'Mycobacterium tuberculosis'

l_query7 = f"""

   match ( b:SmallMolecule ) - [ :hasMolecule ] - ( l:Activity ) - [ :hasAssay ] - ( r:Assay ) - [ :hasTarget ] - ( t:SingleProtein )
   where t.{l_var1}  = '{l_var2}'

   return count(b)
   """

l_result7 = my_graph.query(l_query7)

print(l_result7)



          0/? [?op/s]

   count(b)
0     12955
